# LongT5 finetuning with color-text-size dataset

Input is too long for T5's 512 max input sequence length.

In [1]:
import shutil
!pip install -U git+https://github.com/huggingface/transformers.git@13493215abceafc1653af88b045120014fb4c1fc
!pip install -U git+https://github.com/huggingface/accelerate.git@8ade23cc6aec7c3bd3d80fef6378cafaade75bbe
!pip install -U git+https://github.com/huggingface/datasets.git@01f91bae037c98f2e05456287bab21470adb8f07
!pip install -U git+https://github.com/huggingface/evaluate.git@55f1bc6e072b05c2d9db1589a07e20f38902b1ec
!pip install -U git+https://github.com/huggingface/safetensors.git@f5839b6aee407652aa3078d91206b618dd84e3c2

!pip install sacrebleu
!pip install pytest-playwright
!playwright install
!pip install matplotlib
!pip install pillow
!pip install torchvision
!pip install lpips

!playwright install-deps  

!pip install -U numpy
!pip install tensorboard

  Cloning https://github.com/huggingface/transformers.git (to revision 13493215abceafc1653af88b045120014fb4c1fc) to /tmp/pip-req-build-elj3634y
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-elj3634y
  Running command git rev-parse -q --verify 'sha^13493215abceafc1653af88b045120014fb4c1fc'
  Running command git fetch -q https://github.com/huggingface/transformers.git 13493215abceafc1653af88b045120014fb4c1fc
  Running command git checkout -q 13493215abceafc1653af88b045120014fb4c1fc
  Resolved https://github.com/huggingface/transformers.git to commit 13493215abceafc1653af88b045120014fb4c1fc
  Installing build dependencies ... done
  Getting requirements to build wheel ... one
  Preparing metadata (pyproject.toml) ... one
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 574.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 1.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━

In [2]:
!apt install zip -y
!rm -rf data-rb-color-text-size
!mkdir -p data-rb-color-text-size
!wget "https://www.dropbox.com/scl/fi/689uan4tngw5z1b38hlgt/data-rb-size-color-text.zip?rlkey=tpl5lin2hh2vyn5k3c4dcdanw&dl=1" -O model.zip
!unzip model.zip -d data-rb-color-text-size

!mkdir -p data-rb-validate

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  unzip
The following NEW packages will be installed:
  unzip zip
0 upgraded, 2 newly installed, 0 to remove and 51 not upgraded.
Need to get 350 kB of archives.
After this operation, 930 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 unzip amd64 6.0-26ubuntu3.2 [175 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 zip amd64 3.0-12build2 [176 kB]
Fetched 350 kB in 2s (144 kB/s)m
debconf: delaying package configuration, since apt-utils is not installed

Selecting previously unselected package unzip.
(Reading database ... 34449 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-26ubuntu3.2_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 11%] [######....................................

In [3]:
from datasets import load_from_disk, DatasetDict

validation_size = 4
test_size = 100

dataset = load_from_disk('data-rb-color-text-size')

train_test_split = dataset.train_test_split(test_size=test_size/len(dataset))
train_valid_split = train_test_split['train'].train_test_split(test_size=validation_size/len(dataset))

dataset = DatasetDict({
    "train": train_valid_split["train"],
    "valid": train_valid_split["test"],
    "test": train_test_split["test"],
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['svg', 'html'],
        num_rows: 99749
    })
    valid: Dataset({
        features: ['svg', 'html'],
        num_rows: 4
    })
    test: Dataset({
        features: ['svg', 'html'],
        num_rows: 100
    })
})


In [4]:
print(dataset['train'][0])

{'svg': '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="393" height="852" viewBox="0 0 393 852"><g id="html1"><g data-tag="head" id="head1" data-z-index="auto" data-stacking-context="true" aria-owns="script1"><g data-tag="script" id="script1" data-z-index="auto" data-stacking-context="true"/></g><g data-tag="body" id="body1" data-z-index="auto" data-stacking-context="true" role="document" aria-owns="top bottom style1"><g data-tag="div" id="top" data-z-index="auto" data-stacking-context="true" aria-owns="top-left top-right"><g data-stacking-layer="rootBackgroundAndBorders"><rect width="393" height="460.078125" x="0" y="0" fill="rgb(152, 245, 140)"/></g><g data-tag="div" id="top-left" data-z-index="auto" data-stacking-context="true"><g data-stacking-layer="rootBackgroundAndBorders"><rect width="312.90625" height="460.078125" x="0" y="0" fill="rgb(167, 181, 144)"/></g><text color="rgb(197, 96, 75)" dominant-baseline="text-after-edge" font-family=

In [5]:
from transformers import AutoTokenizer
from tokenizers.processors import TemplateProcessing

tokenizer = AutoTokenizer.from_pretrained("google/long-t5-tglobal-base")
new_words = ['<', '>', '{', '}', '<body>', '<div>', '</div>', '<div', '<style>', '</style>', '</body>']
tokenizer.add_tokens(new_words)

config.json:   0%|          | 0.00/851 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

11

In [6]:
tokenized_output = tokenizer.encode(dataset['train'][0]['svg'], add_special_tokens=False)
unk_token_id = tokenizer.unk_token_id
if unk_token_id in tokenized_output:
    print("Alert: The tokenized SVG contains unknown tokens.")
else:
    print("No unknown tokens found in the SVG. Length: " + str(len(tokenized_output)))

tokenized_output = tokenizer.encode(dataset['train'][0]['html'], add_special_tokens=False)
unk_token_id = tokenizer.unk_token_id
if unk_token_id in tokenized_output:
    print("Alert: The tokenized HTML contains unknown tokens.")
else:
    print("No unknown tokens found in the HTML. Length: " + str(len(tokenized_output)))

No unknown tokens found in the SVG. Length: 1890
No unknown tokens found in the HTML. Length: 301


In [7]:
print(tokenizer.decode(tokenizer.encode(dataset['train'][0]['html'], add_special_tokens=False)))

<body><div id="top"><div id="top-left"> UNDER</div><div id="top-right"> STEAM</div></div><div id="bottom"><div id="bottom-left"> MOUNT</div><div id="bottom-right"> COACH</div></div></body><style> body{ margin: 0; display: flex; flex-direction: column; min-height: 100vh; font-weight: bold;} #top{ background-color: #98f58c; flex-basis: 54vh; flex-grow: 0; flex-shrink: 0; color: #d526b7; font-size: 22px;} #bottom{ background-color: #c049f6; flex: 1 1 auto; color: #feac8e; font-size: 20pt;} #top, #bottom{ display: flex;} #top-left{ flex-basis: 50vw; background: #a7b590; color: #c5604b; font-size: 4em;} #bottom-left{ flex-basis: 210px; background: #d21659; color: #4c330a; font-size: 102%;}</style>


In [8]:
MAX_SEQUENCE_LENGTH = 2048

def get_token_lengths(examples):
    inputs = tokenizer(
        examples['svg'],
        truncation=False,  # Don't truncate yet
        padding=False,     # Don't pad yet
        return_length=True,
    )
    labels = tokenizer(
        examples['html'],
        truncation=False,
        padding=False,
        return_length=True,
    )

    inputs["input_length"] = inputs["length"]
    inputs["labels"] = labels["input_ids"]
    inputs["label_length"] = labels["length"]

    inputs.pop("length", None)

    return inputs

tokenized_data = dataset.map(get_token_lengths, batched=True)

def filter_function(example):
    return example['input_length'] <= MAX_SEQUENCE_LENGTH and example['label_length'] <= MAX_SEQUENCE_LENGTH

filtered_data = tokenized_data.filter(filter_function)

def tokenize_examples(examples):
    inputs = tokenizer(
        examples['svg'],
        max_length=MAX_SEQUENCE_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
    )
    labels = tokenizer(
        examples['html'],
        max_length=MAX_SEQUENCE_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
    )

    labels = labels['input_ids']
    labels[labels == tokenizer.pad_token_id] = -100
    inputs["labels"] = labels
    

    return inputs

filtered_data = filtered_data.map(get_token_lengths, batched=True, remove_columns=dataset["train"].column_names)

print(filtered_data)

Map:   0%|          | 0/99749 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/99749 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/99749 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'input_length', 'labels', 'label_length'],
        num_rows: 99749
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'input_length', 'labels', 'label_length'],
        num_rows: 4
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'input_length', 'labels', 'label_length'],
        num_rows: 100
    })
})


In [9]:
import numpy as np
from utils.similarity import calculate_metrics
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import torch
from types import SimpleNamespace
import shutil

VIEWPORT_SIZES = {
    'DESKTOP': {'width': 1440, 'height': 900},
    'TABLET': {'width': 834, 'height': 1210},
    'MOBILE': {'width': 393, 'height': 852},
}

VIEWPORTS = ['MOBILE']

def add_image_to_tensorboard(name, step, img_path):
    image = Image.open(img_path)
    image = image.convert('RGB')
    image_array = np.array(image)
    image_tensor = torch.from_numpy(image_array)
    image_tensor = image_tensor.permute(2, 0, 1)
    image_tensor = image_tensor.float() / 255.0
    
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_image(name, image_tensor, step)
    
def add_text_to_tensorboard(name, step, text):
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_text(name, text, step)

def postprocess_text(preds, labels):
    preds = [pred.strip().replace('<unk>', '') for pred in preds]
    labels = [[label.strip().replace('<unk>', '')] for label in labels]

    return preds, labels

def create_compute_metrics(trainer, save_best_model_dir = None):
    best_perceptual_loss = None

    def compute_metrics(eval_predictions, to_tensorboard = True):
        nonlocal best_perceptual_loss

        predictions, labels = eval_predictions
        
        if isinstance(predictions, tuple):
            predictions = predictions[0]

        decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_predictions, decoded_labels = postprocess_text(decoded_predictions, decoded_labels)
    
        similarity_scores = []
        perceptual_losses = []
        index = 1
        for prediction, label in zip(decoded_predictions, decoded_labels):
            if to_tensorboard:
                add_text_to_tensorboard(f'valid_{index}_label_text', trainer.state.global_step, label[0])
                add_text_to_tensorboard(f'valid_{index}_prediction_text', trainer.state.global_step, prediction)

            for viewport in VIEWPORTS:
            
                metrics = calculate_metrics(prediction, label[0],
                                        VIEWPORT_SIZES[viewport]['width'], VIEWPORT_SIZES[viewport]['height'])

                if metrics is not None:
                    similarity_scores.append(metrics['similarity'])
                    perceptual_losses.append(metrics['perceptual_loss'])

                    if to_tensorboard:
                        add_image_to_tensorboard(f'valid_{index}_expectation_' + viewport.lower(), trainer.state.global_step, metrics['expected_screenshot_path'])
                        add_image_to_tensorboard(f'valid_{index}_prediction_' + viewport.lower(), trainer.state.global_step, metrics['predicted_screenshot_path'])
            
            index += 1


        result = {
            'similarity': float(np.mean(similarity_scores)),
            'perceptual_loss': float(np.mean(perceptual_losses)),
        }

        if save_best_model_dir is not None:
            if best_perceptual_loss is None or result['perceptual_loss'] < best_perceptual_loss:
                if os.path.exists(save_best_model_dir):
                    shutil.rmtree(save_best_model_dir)
                trainer.save_model(save_best_model_dir)
                tokenizer.save_pretrained(save_best_model_dir)
                with open(os.path.join(save_best_model_dir, 'best_results.txt'), 'w') as f:
                    f.write(str(result) + '\nStep: ' + str(trainer.state.global_step))

                best_perceptual_loss = result['perceptual_loss']
    
        result = {k: round(v, 4) for k, v in result.items()}
        return result
    
    return compute_metrics

def eval_outside_of_training(model, dataset, to_tensorboard = False, step = None):
    def pad_to_matrix(seqs, pad_value=0):
        max_len = max(len(s) for s in seqs)
        matrix  = np.full((len(seqs), max_len), pad_value, dtype=np.int64)
        for i, s in enumerate(seqs):
            matrix[i, :len(s)] = s                 # left-align, pad on the right
        return matrix
    
    model.eval()
    model.to('cuda:0')

    predictions = []
    labels = []
    for row in dataset:
        inputs = tokenizer(row['svg'], return_tensors="pt").input_ids
        labels.append(tokenizer(row['html'], return_tensors="pt").input_ids)
        inputs = inputs.to('cuda:0')
        outputs = model.generate(inputs)
        predictions.append(outputs[0])

    mock_trainer = SimpleNamespace(state=SimpleNamespace(global_step=step))
    compute_metrics = create_compute_metrics(mock_trainer)
    return compute_metrics((
            pad_to_matrix([t.cpu().numpy() for t in predictions]), 
            pad_to_matrix([t.squeeze(0).cpu().numpy() for t in labels])
        ), to_tensorboard=to_tensorboard)



In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="google/long-t5-tglobal-base")

In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoConfig

model = AutoModelForSeq2SeqLM.from_pretrained("google/long-t5-tglobal-base")
model.resize_token_embeddings(len(tokenizer))

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Embedding(32110, 768)

In [12]:
from transformers import GenerationConfig

generation_config = GenerationConfig.from_model_config(model.config)
generation_config._from_model_config = False
generation_config.max_new_tokens = 1024

generation_config

GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "max_new_tokens": 1024,
  "pad_token_id": 0
}

In [13]:
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_output_dir = f"model-{timestamp}"
log_dir = f'{model_output_dir}/runs'

In [14]:
results = eval_outside_of_training(model, dataset["valid"], True, 0)
writer = SummaryWriter(log_dir=log_dir)   # or reuse the one you have
for k, v in results.items():
    writer.add_scalar(f"eval/{k}", v, 0)

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [15]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import os

training_args = Seq2SeqTrainingArguments(
    output_dir=model_output_dir,
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    logging_steps=25,
    logging_dir=log_dir,
    predict_with_generate=True,
    # load_best_model_at_end=True,
    bf16=True,
    generation_config=generation_config,
    report_to="tensorboard",
    save_safetensors=False # Fixing missing encoder.embed_tokens.weight and decoder.embed_tokens.weight due to a bug.
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=filtered_data["train"],
    eval_dataset=filtered_data["valid"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=None,
)

trainer.compute_metrics = create_compute_metrics(trainer, os.path.join(model_output_dir, "best"))

trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss,Similarity,Perceptual Loss,Runtime,Samples Per Second,Steps Per Second
1000,0.549800,0.494097,0.906800,0.573300,76.179400,0.053000,0.013000
2000,0.522000,0.487397,0.885300,0.598300,73.208500,0.055000,0.014000
3000,0.503900,0.470329,0.893500,0.567400,75.539100,0.053000,0.013000
4000,0.424100,0.351670,0.944900,0.283900,76.686800,0.052000,0.013000
5000,0.220600,0.137064,0.993900,0.070900,75.360700,0.053000,0.013000
6000,0.147200,0.088586,0.999500,0.016300,75.388000,0.053000,0.013000
7000,0.112000,0.071206,0.999300,0.012800,75.742500,0.053000,0.013000
8000,0.091900,0.054379,0.998600,0.022900,72.416500,0.055000,0.014000
9000,0.075300,0.036698,0.999900,0.002700,75.786100,0.053000,0.013000
10000,0.061400,0.025907,0.999900,0.002700,72.181100,0.055000,0.014000


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/opt/conda/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: Futu

TrainOutput(global_step=62340, training_loss=0.07453208561600651, metrics={'train_runtime': 94686.6912, 'train_samples_per_second': 10.535, 'train_steps_per_second': 0.658, 'total_flos': 2.5447318218391265e+18, 'train_loss': 0.07453208561600651, 'epoch': 9.99847622102815})

In [16]:
import os

trainer.save_model(os.path.join(model_output_dir, "final"))
tokenizer.save_pretrained(os.path.join(model_output_dir, "final"))

('model-2025-04-24_14-50-02/final/tokenizer_config.json',
 'model-2025-04-24_14-50-02/final/special_tokens_map.json',
 'model-2025-04-24_14-50-02/final/tokenizer.json')

In [17]:
from transformers import LongT5ForConditionalGeneration

text = dataset['test'][0]['svg']

inputs = tokenizer(text, return_tensors="pt").input_ids
inputs = inputs.to('cuda:0')

model = LongT5ForConditionalGeneration.from_pretrained(os.path.join(model_output_dir, "best"))
model.to('cuda:0')
model.generation_config = generation_config

outputs = model.generate(inputs)
# outputs = model.generate(inputs, max_new_tokens=16_384, do_sample=True, top_k=30, top_p=0.95)

translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(dataset['test'][0]['html'])
print(translation)


<body><div id="top"><div id="top-left">THROW</div><div id="top-right">CHOSE</div></div><div id="bottom"><div id="bottom-left">WATER</div><div id="bottom-right">ANGLE</div></div></body>

<style>


        body {
            margin: 0;
            display: flex;
            flex-direction: column;
            min-height: 100vh;
            font-weight: bold;
        }

        #top {
            background-color: #27ed9e;
            flex-basis: 66vh;
            flex-grow: 0;
            flex-shrink: 0;
            color: #1f7b33;
            font-size: 27px;
        }

        #bottom {
            background-color: #ab94d6;
            flex: 1 1 auto;
            color: #fec1f7;
            font-size: 17pt;
        }

        #top, #bottom {
            display: flex;
        }

        #top-left {
            flex-basis: 40vw;
            background: #801426;
            color: #5fc476;
            font-size: 3em;
        }

        #bottom-left {
            flex-basis: 172px;
     

In [18]:
test_results = eval_outside_of_training(model, dataset["test"], False)

print(test_results)

{'similarity': 0.9999, 'perceptual_loss': 0.0015}
